In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from tensorflow.keras.callbacks import TensorBoard
log_folder = 'logs'
callbacks = [TensorBoard(log_dir=log_folder,
                         histogram_freq=1,
                         write_graph=True,
                         write_images=True,
                         update_freq='epoch',
                         profile_batch=1,
                         embeddings_freq=1)]
# Load data
X, y = load_iris(return_X_y=True)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Find centers using K-means clustering
num_centers = 3
kmeans = KMeans(n_clusters=num_centers, random_state=42).fit(X_train)
centers = kmeans.cluster_centers_

# Define RBF layer
class RBFLayer(keras.layers.Layer):
    def __init__(self, num_centers):
        super(RBFLayer, self).__init__()
        self.num_centers = num_centers
        self.centers = self.add_weight(name='centers', shape=(self.num_centers, X_train.shape[1]), initializer='uniform', trainable=True)
        self.widths = self.add_weight(name='widths', shape=(self.num_centers,), initializer='ones', trainable=True)
        self.linear = layers.Dense(units=1, activation=None)
    
    def radial(self, X):
        X = tf.expand_dims(X, axis=1)
        centers = tf.expand_dims(self.centers, axis=0)
        dist = tf.reduce_sum(tf.square(X - centers), axis=-1)
        return tf.exp(-dist / (2 * tf.square(self.widths)))
    
    def call(self, X):
        radial_output = self.radial(X)
        return self.linear(radial_output)

# Define RBF network
model = keras.Sequential([
    RBFLayer(num_centers),
])

# Compile model
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=300, batch_size=128, verbose=2,callbacks=callbacks)

# Evaluate model on test set
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {test_acc:.4f}")


2023-03-23 15:43:58.623105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-23 15:43:59.332097: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-03-23 15:43:59.332193: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib/cuda/include:/usr/lib/cuda/lib64:
2023-03-23 15:43:59.332200: W tensorflow/com

Epoch 1/300


2023-03-23 15:44:00.547180: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-23 15:44:00.554383: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-23 15:44:00.554880: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-23 15:44:00.555220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, 

1/1 - 2s - loss: 1.4343 - accuracy: 0.3333 - 2s/epoch - 2s/step
Epoch 2/300


2023-03-23 15:44:02.999295: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2023-03-23 15:44:03.000125: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:140] cuptiFinalize: ignored due to a previous error.
2023-03-23 15:44:03.000182: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:457] cuptiGetResultString: ignored due to a previous error.
2023-03-23 15:44:03.000191: E tensorflow/core/profiler/backends/gpu/cupti_tracer.cc:1807] function cupti_interface_->Finalize()failed with error 
2023-03-23 15:44:03.003125: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:133] cuptiGetTimestamp: ignored due to a previous error.
2023-03-23 15:44:03.003183: E tensorflow/core/profiler/backends/gpu/cupti_error_manager.cc:133] cuptiGetTimestamp: ignored due to a previous error.
2023-03-23 15:44:03.003193: I tensorflow/core/profiler/backends/gpu/cupti_collector.cc:522]  GpuTracer has collected 0 callback api events and 0 a

1/1 - 0s - loss: 1.4275 - accuracy: 0.3333 - 74ms/epoch - 74ms/step
Epoch 3/300
1/1 - 0s - loss: 1.4206 - accuracy: 0.3333 - 72ms/epoch - 72ms/step
Epoch 4/300
1/1 - 0s - loss: 1.4138 - accuracy: 0.3333 - 74ms/epoch - 74ms/step
Epoch 5/300
1/1 - 0s - loss: 1.4071 - accuracy: 0.3333 - 75ms/epoch - 75ms/step
Epoch 6/300
1/1 - 0s - loss: 1.4003 - accuracy: 0.3333 - 72ms/epoch - 72ms/step
Epoch 7/300
1/1 - 0s - loss: 1.3936 - accuracy: 0.3333 - 84ms/epoch - 84ms/step
Epoch 8/300
1/1 - 0s - loss: 1.3869 - accuracy: 0.3333 - 81ms/epoch - 81ms/step
Epoch 9/300
1/1 - 0s - loss: 1.3803 - accuracy: 0.3333 - 74ms/epoch - 74ms/step
Epoch 10/300
1/1 - 0s - loss: 1.3736 - accuracy: 0.3333 - 73ms/epoch - 73ms/step
Epoch 11/300
1/1 - 0s - loss: 1.3670 - accuracy: 0.3333 - 73ms/epoch - 73ms/step
Epoch 12/300
1/1 - 0s - loss: 1.3604 - accuracy: 0.3333 - 77ms/epoch - 77ms/step
Epoch 13/300
1/1 - 0s - loss: 1.3538 - accuracy: 0.3333 - 74ms/epoch - 74ms/step
Epoch 14/300
1/1 - 0s - loss: 1.3473 - accuracy:

Epoch 104/300
1/1 - 0s - loss: 0.8841 - accuracy: 0.6000 - 85ms/epoch - 85ms/step
Epoch 105/300
1/1 - 0s - loss: 0.8803 - accuracy: 0.6000 - 95ms/epoch - 95ms/step
Epoch 106/300
1/1 - 0s - loss: 0.8765 - accuracy: 0.6000 - 93ms/epoch - 93ms/step
Epoch 107/300
1/1 - 0s - loss: 0.8727 - accuracy: 0.6000 - 97ms/epoch - 97ms/step
Epoch 108/300
1/1 - 0s - loss: 0.8689 - accuracy: 0.6000 - 96ms/epoch - 96ms/step
Epoch 109/300
1/1 - 0s - loss: 0.8652 - accuracy: 0.6000 - 97ms/epoch - 97ms/step
Epoch 110/300
1/1 - 0s - loss: 0.8615 - accuracy: 0.6000 - 96ms/epoch - 96ms/step
Epoch 111/300
1/1 - 0s - loss: 0.8578 - accuracy: 0.6000 - 94ms/epoch - 94ms/step
Epoch 112/300
1/1 - 0s - loss: 0.8541 - accuracy: 0.6000 - 95ms/epoch - 95ms/step
Epoch 113/300
1/1 - 0s - loss: 0.8505 - accuracy: 0.6000 - 84ms/epoch - 84ms/step
Epoch 114/300
1/1 - 0s - loss: 0.8469 - accuracy: 0.6083 - 102ms/epoch - 102ms/step
Epoch 115/300
1/1 - 0s - loss: 0.8433 - accuracy: 0.6083 - 97ms/epoch - 97ms/step
Epoch 116/300


Epoch 204/300
1/1 - 0s - loss: 0.6040 - accuracy: 0.6500 - 86ms/epoch - 86ms/step
Epoch 205/300
1/1 - 0s - loss: 0.6020 - accuracy: 0.6500 - 84ms/epoch - 84ms/step
Epoch 206/300
1/1 - 0s - loss: 0.6001 - accuracy: 0.6500 - 83ms/epoch - 83ms/step
Epoch 207/300
1/1 - 0s - loss: 0.5982 - accuracy: 0.6500 - 83ms/epoch - 83ms/step
Epoch 208/300
1/1 - 0s - loss: 0.5963 - accuracy: 0.6500 - 84ms/epoch - 84ms/step
Epoch 209/300
1/1 - 0s - loss: 0.5944 - accuracy: 0.6500 - 88ms/epoch - 88ms/step
Epoch 210/300
1/1 - 0s - loss: 0.5925 - accuracy: 0.6500 - 84ms/epoch - 84ms/step
Epoch 211/300
1/1 - 0s - loss: 0.5906 - accuracy: 0.6500 - 84ms/epoch - 84ms/step
Epoch 212/300
1/1 - 0s - loss: 0.5888 - accuracy: 0.6500 - 86ms/epoch - 86ms/step
Epoch 213/300
1/1 - 0s - loss: 0.5869 - accuracy: 0.6500 - 81ms/epoch - 81ms/step
Epoch 214/300
1/1 - 0s - loss: 0.5851 - accuracy: 0.6500 - 83ms/epoch - 83ms/step
Epoch 215/300
1/1 - 0s - loss: 0.5833 - accuracy: 0.6500 - 84ms/epoch - 84ms/step
Epoch 216/300
1/

In [3]:
%load_ext tensorboard
# %reload_ext tensorboard
log_folder = './logs'
%tensorboard --logdir={log_folder}

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
